# Amazon Forecast  Sample Inventory Planning

Sample INVENTORY_PLANNING for  bike rentals. The bike rental data came from the [Kaggle Bike Share dataset](https://www.kaggle.com/c/bike-sharing-demand) 

Prerequisites: 
[AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/installing.html) . 

For more informations about APIs, please check the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html)

## Table Of Contents
* [Setting up](#setup)
* [Test Setup - Running first API](#hello)
* [Forecasting Example with Amazon Forecast](#forecastingExample)

**Read Every Cell FULLY before executing it**


## Set up Preview SDK<a class="anchor" id="setup"></a>

In [ ]:
# Configures your AWS CLI to now understand our up and coming service Amazon Forecast
!aws configure add-model --service-model file://../sdk/forecastquery-2018-06-26.normal.json --service-name forecastquery
!aws configure add-model --service-model file://../sdk/forecast-2018-06-26.normal.json --service-name forecast

In [ ]:
# Prerequisites : 1 time install only, remove the comments to execute the lines.
#!pip install boto3
#!pip install pandas

In [ ]:
import boto3
from time import sleep
import subprocess

In [ ]:
session = boto3.Session(region_name='us-west-2') #us-east-1 is also supported

forecast = session.client(service_name='forecast')
forecastquery = session.client(service_name='forecastquery')

##  Setup <a class="anchor" id="hello"></a>
We will confirm that the notebook can access  Amazon Forecast and  interact with our Simple API ListRecipes. The API returns a list of the global recipes Forecast offers that you could potentially use as a part of your forecasting solution. 

In [ ]:
forecast.list_recipes()

*If this ran successfully, kudos! If there are any errors at this point runing the following list_recipes, please contact us at the [AWS support forum](https://forums.aws.amazon.com/forum.jspa?forumID=327)

## Forecasting with Amazon Forecast<a class="anchor" id="forecastingExample"></a>
### Preparing your Data

In Amazon Forecast , a dataset is a collection of file(s) which contain data that is relevant for a forecasting task. A dataset must conform to a schema provided by Amazon Forecast. 

For this exercise, we use sample bike share rental dataset. 

# Data Pre-processing 
  Amazon forecast can import data from Amazon S3. We first explore the data locally to see the attributes . We are using all the attriutes , so the user can modify the attributes to see the different results. 
  IF you remove any attributes , those changes must be reflected in 

In [ ]:
import pandas as pd
trf = pd.read_csv("../data/train.csv", dtype = object)
trf['count'] = trf['count'].astype(float) 
trf['itemname'] = 'bike_12'
trf.to_csv("../data/bike.csv", index= False, header = False)
trf.head(3)

In [ ]:
df = pd.read_csv("../data/bike.csv", dtype = object)
df.head(3)


Now upload the data to S3. But before doing that, go into your AWS Console, select S3 for the service and create a new bucket inside the `Oregon` or `us-west-2` region. Use that bucket name convention of `amazon-forecast-unique-value-data`. The name must be unique, if you get an error, just adjust until your name works, then update the `bucketName` cell below.

In [ ]:
s3 = session.client('s3')

In [ ]:
accountId = boto3.client('sts').get_caller_identity().get('Account')

In [ ]:
bucketName = "amazon-forecast-data-{0}".format(accountId)# Update the unique-value bit here.
key="bikedata/bike.csv"

In [ ]:
s3.upload_file(Filename="../data/bike.csv", Bucket=bucketName, Key=key)

In [ ]:
bucketName

In [ ]:
# One time setup only, uncomment the following command to create the role to provide to Amazon Forecast. 
# Save the generated role for all future calls to use for importing or exporting data. 

cmd = 'python ../setup_forecast_permissions.py '+bucketName
p = subprocess.Popen(cmd.split(' '), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
roleArn = 'arn:aws:iam::%s:role/amazonforecast'%accountId

### CreateDataset

More details about `Domain` and dataset type can be found on the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/howitworks-domains-ds-types.html) . For this example, we are using [RETAIL](https://docs.aws.amazon.com/forecast/latest/dg/retail-domain.html) domain with 3 required attributes `timestamp`, `demand` and `item_id`. Also for your project name, update it to reflect your name in a lowercase format.

In [ ]:
DATASET_FREQUENCY = "H" 
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

In [ ]:
project = 'bike_forecastdemo' # Replace this with a unique name here, make sure the entire name is < 30 characters.
datasetName= project+'_ds'
datasetGroupName= project +'_gp'
s3DataPath = "s3://"+bucketName+"/"+key
s3DataPath

In [ ]:
datasetName

In [ ]:
### Schema Definition 
### We are defining the attributes for the model 

In [ ]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[    
      { "AttributeName":"timestamp",     "AttributeType":"timestamp"    },      
      { "AttributeName":"season",        "AttributeType":"integer"      },      
      { "AttributeName":"holiday",       "AttributeType":"integer"      },      
      { "AttributeName":"workday",       "AttributeType":"integer"      },      
      { "AttributeName":"weather",       "AttributeType":"integer"      },     
      { "AttributeName":"temperature",   "AttributeType":"float"        },       
      { "AttributeName":"atemp",         "AttributeType":"float"        }, 
      { "AttributeName":"humidity",      "AttributeType":"integer"      },       
      { "AttributeName":"windspeed",     "AttributeType":"float"      },       
      { "AttributeName":"casual",        "AttributeType":"integer"      },      
      { "AttributeName":"registered",    "AttributeType":"integer"      },         
      { "AttributeName":"demand",         "AttributeType":"float"       },      
      { "AttributeName":"item_id",       "AttributeType":"string"       }       
  ]
}

response=forecast.create_dataset(
                    Domain="RETAIL",
                    DatasetType='TARGET_TIME_SERIES',
                    DataFormat='CSV',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    TimeStampFormat=TIMESTAMP_FORMAT,
                    Schema = schema
                   )

In [ ]:
forecast.describe_dataset(DatasetName=datasetName)

In [ ]:
forecast.create_dataset_group(DatasetGroupName=datasetGroupName,RoleArn=roleArn,DatasetNames=[datasetName])

If you have an existing datasetgroup, you can update it

In [ ]:
forecast.describe_dataset_group(DatasetGroupName=datasetGroupName)

### Create Data Import Job
Brings the data into Amazon Forecast system ready to forecast from raw data. 

In [ ]:
ds_import_job_response=forecast.create_dataset_import_job(DatasetName=datasetName,Delimiter=',', DatasetGroupName =datasetGroupName ,S3Uri= s3DataPath)


In [ ]:
ds_versionId=ds_import_job_response['VersionId']
print(ds_versionId)

Check the status of dataset, when the status change from **CREATING** to **ACTIVE**, we can continue to next steps. Depending on the data size. It can take 10 mins to be **ACTIVE**. This process will take 5 to 10 minutes.

In [ ]:
while True:
    dataImportStatus = forecast.describe_dataset_import_job(DatasetName=datasetName,VersionId=ds_versionId)['Status']
    print(dataImportStatus)
    if dataImportStatus != 'ACTIVE' and dataImportStatus != 'FAILED':
        sleep(30)
    else:
        break

In [ ]:
forecast.describe_dataset_import_job(DatasetName=datasetName,VersionId=ds_versionId)

### Recipe

In [ ]:
recipesResponse=forecast.list_recipes()
recipesResponse

Get details about each recipe.

In [ ]:
forecast.describe_recipe(RecipeName='forecast_MQRNN')

### Create Solution 

Forecast horizon is how long in future the forecast should be predicting. For weekly data, a value of 12 means 1 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [ ]:
predictorName= project+'_mqrnn'

In [ ]:
forecastHorizon = 24

In [ ]:
createPredictorResponse=forecast.create_predictor(RecipeName='forecast_MQRNN',DatasetGroupName= datasetGroupName ,PredictorName=predictorName, 
  ForecastHorizon = forecastHorizon)

In [ ]:
predictorVerionId=createPredictorResponse['VersionId']

In [ ]:
forecast.list_predictor_versions(PredictorName=predictorName)

Check the status of solutions, when the status change from **CREATING** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [ ]:
while True:
    predictorStatus = forecast.describe_predictor(PredictorName=predictorName,VersionId=predictorVerionId)['Status']
    print(predictorStatus)
    if predictorStatus != 'ACTIVE' and predictorStatus != 'FAILED':
        sleep(30)
    else:
        break

### Get Error Metrics
 This provides metrics on the accuracy of the models that was  trained with the CreatePredictor operation.
 Use metrics to see how well the model did and to decide whether to use the predictor to generate forecasts 

In [ ]:
forecastquery.get_accuracy_metrics(PredictorName=predictorName)

### Deploy Predictor

In [ ]:
forecast.deploy_predictor(PredictorName=predictorName)

In [ ]:
deployedPredictorsResponse=forecast.list_deployed_predictors()
print(deployedPredictorsResponse)

Please note that the following cell can also take 10 minutes or more to be fully operational. There's no output here, but that is fine as long as the * is there.

In [ ]:
while True:
    deployedPredictorStatus = forecast.describe_deployed_predictor(PredictorName=predictorName)['Status']
    print(deployedPredictorStatus)
    if deployedPredictorStatus != 'ACTIVE' and deployedPredictorStatus != 'FAILED':
        sleep(30)
    else:
        break
print(deployedPredictorStatus)

### Get Forecast

When the solution is deployed and forecast results are ready, you can view them. 
The result set shows the predictions and confidence on the p10,p50 rates. 

In [ ]:
 forecastResponse = forecastquery.get_forecast(
    PredictorName=predictorName,
    Interval="hour",
   Filters={"item_id":"bike_12"}
)
print(forecastResponse)

In [ ]:
## Generate Plot 
print("Forcast for P90") 
d = pd.DataFrame.from_dict(forecastResponse['Forecast'])
df = pd.DataFrame.from_dict(d.loc['mean']['Predictions']).dropna().rename(columns = {'Val':'p90'})
df.plot()

# Export Forecast

You can batch export forecast to s3 bucket. To do so an role with s3 put access is needed, but this has already been created.

In [ ]:
forecastInfoList= forecast.list_forecasts(PredictorName=predictorName)['ForecastInfoList']
forecastId= forecastInfoList[0]['ForecastId']

In [ ]:
outputPath="s3://"+bucketName+"/output"

In [ ]:
forecastExportResponse = forecast.create_forecast_export_job(ForecastId=forecastId, OutputPath={"S3Uri": outputPath,"RoleArn":roleArn})

In [ ]:
forecastExportJobId = forecastExportResponse['ForecastExportJobId']

In [ ]:
while True:
    forecastExportStatus = forecast.describe_forecast_export_job(ForecastExportJobId=forecastExportJobId)['Status']
    print(forecastExportStatus)
    if forecastExportStatus != 'ACTIVE' and forecastExportStatus != 'FAILED':
        sleep(30)
    else:
        break

Check s3 bucket for results

In [ ]:
s3.list_objects(Bucket=bucketName,Prefix="output")

# Cleanup

While Forecast is in preview there are no charges for using it, but to future proof this work below are the instructions to cleanup your work space.

In [ ]:
# Delete Deployed Predictor 
forecast.delete_deployed_predictor(PredictorName=predictorName)

In [ ]:
# Delete the Predictor: 
forecast.delete_predictor(PredictorName=predictorName)

In [ ]:
# Delete Import
forecast.delete_dataset_import(DatasetName=datasetName)

In [ ]:
# Delete Dataset Group
forecast.delete_dataset_group(DatasetGroupName=datasetGroupName)